In [1]:
from xgboost import XGBRegressor
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

In [3]:
#删除outlier
train_df = pd.read_csv('train.csv',header=0,index_col='id',parse_dates=['purchase_date','release_date']).assign(is_train = 1)
train_df = train_df.drop(index=[220,312],axis=0)

In [4]:
X_df = train_df.drop('playtime_forever',axis=1)
y_df = train_df.playtime_forever
train_df.corr().playtime_forever

playtime_forever          1.000000
is_free                  -0.020551
price                     0.125842
total_positive_reviews    0.402915
total_negative_reviews    0.393285
is_train                       NaN
Name: playtime_forever, dtype: float64

In [5]:
test_df = pd.read_csv('test.csv',header=0,index_col='id',parse_dates=['purchase_date','release_date']).assign(is_train = 0)
df = pd.concat([X_df,test_df])

In [6]:
#获取购入时间间隔
df['time_gap'] = (df.purchase_date-df.release_date).apply(lambda x:x.days)

In [7]:
#是否免费数字化
df['is_free'] = np.multiply(df['is_free'],1)

In [8]:
df = df.drop(['purchase_date','release_date'],axis=1)

In [9]:
df = df.fillna(df.mean())

In [10]:
#onehot处理
genres = df['genres'].str.split(',')
genres = genres.str.join(sep=',').str.get_dummies(sep=',')
categories = df.categories.str.get_dummies(',')
tags = df.tags.str.get_dummies(',')

In [11]:
ndf = pd.concat([df.reset_index(drop=True),categories.reset_index(drop=True),genres.reset_index(drop=True),tags.reset_index(drop=True)], axis=1)

In [12]:
ndf = ndf.drop(["genres","categories","tags","is_free"],axis = 1)

In [13]:
#数据处理
standardOPT = lambda x:(x-np.mean(x))/(np.std(x))
normalizationOPT = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
process_cols = ['price', 'total_positive_reviews', 'total_negative_reviews', 'time_gap']

In [14]:
for x in process_cols:
    ndf[x] = ndf[[x]].apply(normalizationOPT)
    ndf[x] = ndf[[x]].apply(standardOPT)

In [16]:
nX_df = ndf[ndf['is_train'] == 1].reset_index().drop("index",axis = 1)
ntest  = ndf[ndf['is_train'] == 0].reset_index().drop("index",axis = 1)
X = nX_df
y = y_df

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=500, random_state=345)
rf.fit(X, y)

accuracies = cross_val_score(estimator = rf, X=X, y=y, cv=3, scoring='neg_mean_squared_error')
rmse_rf = np.sqrt(-accuracies).mean()

In [18]:
rmse_rf

10.179420758185817

In [19]:
y_pred = rf.predict(ntest)
result_rf = pd.DataFrame(y_pred)
result_rf['id']=range(len(result_rf))
result_rf.set_index('id',inplace=True)
result_rf.columns = ['playtime_forever']
result_rf.to_csv("./rf_tag_only.csv",index=True)

In [26]:
rf = RandomForestRegressor()
rf.fit(X, y)

accuracies = cross_val_score(estimator = rf, X=X, y=y, cv=3, scoring='neg_mean_squared_error')
rmse_rf = np.sqrt(-accuracies).mean()
print(rmse_rf)

D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


11.703702188565918


In [27]:
y_pred = rf.predict(ntest)
result_rf = pd.DataFrame(y_pred)
result_rf['id']=range(len(result_rf))
result_rf.set_index('id',inplace=True)
result_rf.columns = ['playtime_forever']
result_rf.to_csv("./rf_none.csv",index=True)